# Task 1
# Your task is to implement a forecasting system as Python script (i.e. a .py file). It should be able to forecast up to 5 time series and dynamically select suitable models for each time series according to some logic. You as expert team are tasked with devising a simple logic.
# At the top of your script, you should allow your users to set some variables as input:
# • file_name: str, file name of data to be read in.
# • ids_to_forecast: list[str], time series names of time series selected to be forecasted.
# • metric: str, either 'mape' or 'mse'.
# If the input for metric is not correct, throw an error and stop the execution.
# Your forecasting system then should do the following:
# a) Read in the data as given in proj1_exampleinput.csv.
# b) Perform any necessary data preparation and filter the data such that you obtain the specified time series.

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import sys  # Needed to exit the script in case of error


# --------------------- USER INPUT ---------------------
file_name = 'proj1_exampleinput.csv'
ids_to_forecast = ['C1002', 'C1050', 'C1080', 'C1016', 'C1126']
metric = 'mape'  # 'mape' or 'mse'
# -----------------------------------------------------

# Validate the metric input
valid_metrics = ['mape', 'mse']
if metric not in valid_metrics:
    raise ValueError(f"❌ Invalid metric '{metric}'. Please choose either 'mape' or 'mse'.")

# Read and process data into Nixtla format
data = pd.read_csv(file_name, parse_dates=['Month'])

# Filter to include only the selected time series
data = data[data['product_class'].isin(ids_to_forecast)]

# Sort by ID and timestamp
data = data.sort_values(by=['product_class', 'Month'])

# Rename columns to match Nixtla format
data = data.rename(columns={'product_class': 'unique_id', 'Month': 'ds', 'sales_volume': 'y'})

# Reset index for clean structure
data = data.reset_index(drop=True)

# c) Select the models for forecasting using the following logic. For each category - if not explicitly specified differently - use 2-4 suitable models from the lecture:
# • Category 1: Time series is shorter than two seasonality periods (pick only one model).
# • Category 2: Else, time series contains zeroes and has high seasonality.
# • Category 3: Else, time series has high seasonality and only positive values.
# • Category 4: Times series has low or no seasonality and zeroes.
# • Category 5: All remaining.
# • Additionally, in all cases always run the Naive forecast as benchmark.

In [2]:
from statsforecast.models import (
    Naive,
    HistoricAverage,
    RandomWalkWithDrift,
    SeasonalNaive,
    SeasonalWindowAverage,
    AutoETS,
    HoltWinters,
    AutoARIMA
)


# --------------------- Model selection by category ---------------------
def has_high_seasonality(series):
    result = seasonal_decompose(series, model='additive', period=12, extrapolate_trend='freq')
    seasonal_var = np.var(result.seasonal.dropna())
    resid_var = np.var(result.resid.dropna())
    return seasonal_var > resid_var


def is_positive(series):
    return series.min() > 0

def has_zeroes(series):
    return (series == 0).any()

def get_category(ts):
    if len(ts) < 24:
        return 1
    elif has_zeroes(ts) and has_high_seasonality(ts):
        return 2
    elif has_high_seasonality(ts) and is_positive(ts):
        return 3
    elif not has_high_seasonality(ts) and has_zeroes(ts):
        return 4
    else:
        return 5


# Selected models by category with justification:
category_models = {
    1: [
        # Category 1: Shorter than two seasonality periods
        # Models:
        # - Naive: Serves as a baseline for all forecasting tasks. It assumes the next value is the last observed one. Particularly useful when data is limited.
        # - HistoricAverage: Calculates the mean of historical values. Ideal when no seasonal or trend patterns are observable due to short history.
        # - Reasoning: Since there is not enough data, complex models may not be able to capture seasonality or trends optimally. Simple models can work with short ranges.
        
        Naive(),
        HistoricAverage()
    ],
    2: [
        # Category 2: High seasonality + zero values
        # Models:
        # - Naive: Always included as a Benchmark.
        # - SeasonalNaive: Projects the value from the same point in the previous season. Strong benchmark when seasonality dominates.
        # - SeasonalWindowAverage: Smooths seasonal patterns by averaging same-period values from previous seasons, reducing variance.
        # - SARIMA: Handles both seasonality and trend components. Can model autoregression, moving averages, and seasonal differencing.
        # - Holt-Winters (Additive): Suitable for capturing trend and seasonality when values can reach zero — additive form avoids issues that multiplicative models would face with zeroes.
        # - Reasoning: When seasonality is present and the data includes zero values (e.g., sales data with no sales in some periods), additive models are safer than multiplicative ones, which assume strictly positive values.
        # The SeasonalNaive and SeasonalWindowAverage models exploit the strong seasonal signals, while SARIMA and Additive Holt-Winters are more robust and flexible for detecting seasonal patterns and underlying trends.

        Naive(),
        SeasonalNaive(season_length=12),
        SeasonalWindowAverage(window_size=3, season_length=12),
        AutoARIMA(season_length=12, alias='SARIMA'),
        HoltWinters(season_length=12, error_type='A', alias='HW_Add')  # Additive
    ],
    3: [
        # Category 3: High seasonality + positive values
        # Models:
        # - Naive: Always included as a Benchmark.
        # - SeasonalNaive: A reliable baseline for data with regular seasonality.
        # - SeasonalWindowAverage: Averages values from the same season across previous cycles to smooth irregularities.
        # - SARIMA: Capable of capturing both trend and seasonality via seasonal ARIMA formulation.
        # - AutoETS: Automatically selects between additive and multiplicative exponential smoothing. Since values are strictly positive, multiplicative Holt-Winters can be considered, which better captures proportional seasonal effects.
        # - Reasoning: With strong seasonality and no zero values, multiplicative models become viable. They model seasonal effects as a proportion of the level (e.g., summer sales being 1.5× baseline).
        # AutoETS explores this automatically. SARIMA is useful when autocorrelation structures are complex. Combined, these models cover a spectrum from simplicity (Naive) to flexibility (SARIMA/AutoETS).

        Naive(),
        SeasonalNaive(season_length=12),
        SeasonalWindowAverage(window_size=3, season_length=12),
        AutoARIMA(season_length=12, alias='SARIMA'),
        AutoETS(model='ZZZ', alias='AutoETS')  # Includes additive and multiplicative HW
    ],
    4: [
        # Category 4: Low seasonality + zero values
        # Models:
        # - Naive: Always included as a Benchmark.
        # - HistoricAverage: Effective when values fluctuate around a stable mean.
        # - RandomWalkWithDrift: Assumes the series follows a trend with random variation. Useful when there's no seasonality but a clear directional movement.
        # - AutoETS (Simple/Double Exponential Smoothing): Simple ES assumes no trend/seasonality; double ES introduces trend while maintaining simplicity.
        # - ARIMA: General-purpose model that can capture linear trends and autocorrelation without requiring seasonality.
        # - Reasoning: In the absence of strong seasonality, models that focus on level and trend (rather than seasonal cycles) are preferred.
        # If data is noisy or exhibits random walk behavior, RWWD and ARIMA are excellent choices. AutoETS allows automatic selection between simple and double exponential smoothing.
        # HistoricAverage acts as a fallback for stationary, mean-reverting series.

        Naive(),
        HistoricAverage(),
        RandomWalkWithDrift(),
        AutoETS(model='ZAN', alias='AutoETS_SimpleDouble'),
        AutoARIMA(season_length=1, alias='ARIMA')
    ],
    5: [
        # Category 5: All other cases
        # Models:
        # - Naive: Always included for benchmarking.
        # - ARIMA: Adaptable model for data with autocorrelation and trend, without requiring seasonality.
        # - AutoETS (Simple/Double, damped trend): Offers flexibility across level and trend modeling with exponential smoothing, including damped trend components for stabilizing long-term forecasts.
        # - HistoricAverage: Provides robustness in noisy datasets or when structure is unclear.
        # - RandomWalkWithDrift: Simple trend model, less prone to overfitting, useful when trend is present but weak.
        # - Reasoning: When the structure of the series is uncertain or doesn't fit neatly into seasonal/non-seasonal classifications, we need versatile models. ARIMA and AutoETS can adapt to various patterns.
        # Damped trends are especially useful when the series appears to trend but might plateau. Simpler models like Naive, HistoricAverage, and RWWD help prevent overfitting and offer interpretability.
        
        Naive(),
        AutoARIMA(season_length=12, alias='ARIMA'),
        AutoETS(model='ZAN', alias='AutoETS_SimpleDouble'),
        HistoricAverage(),
        RandomWalkWithDrift()
    ]
}

# Assign models to each time series based on its category
ts_models = {}
for uid in data['unique_id'].unique():
    ts = data[data['unique_id'] == uid]['y']
    cat = get_category(ts)
    ts_models[uid] = category_models[cat]
    print(f"Series {uid} assigned to Category {cat} with models: {[type(m).__name__ for m in ts_models[uid]]}")

Series C1002 assigned to Category 5 with models: ['Naive', 'AutoARIMA', 'AutoETS', 'HistoricAverage', 'RandomWalkWithDrift']
Series C1016 assigned to Category 5 with models: ['Naive', 'AutoARIMA', 'AutoETS', 'HistoricAverage', 'RandomWalkWithDrift']
Series C1050 assigned to Category 1 with models: ['Naive', 'HistoricAverage']
Series C1080 assigned to Category 3 with models: ['Naive', 'SeasonalNaive', 'SeasonalWindowAverage', 'AutoARIMA', 'AutoETS']
Series C1126 assigned to Category 5 with models: ['Naive', 'AutoARIMA', 'AutoETS', 'HistoricAverage', 'RandomWalkWithDrift']


# d) Perform a cross validation simulating forecasts for one year, every half year for an evaluation period of 2 years. Do not perform cross-validation for time series shorter than 48 observations.
# e) Calculate the accuracy for all models over all errors with the metric chosen by the user. If at least one value of a time series is zeroes, throw a warning if MAPE is used.
# f) The system should now select the best performing single forecasting method per time series according to the metric.

In [3]:
import pandas as pd
import numpy as np
import warnings
from statsforecast import StatsForecast
from utilsforecast.evaluation import evaluate
import utilsforecast.losses as ufl
from statsforecast.models import Naive

# Define key parameters
forecast_horizon = 12          # Number of periods to forecast
min_obs = 48                   # Minimum number of observations required to apply evaluation
rolling_step = 6               # Step size for rolling evaluation
eval_window = 24               # Total number of test observations (split into rolling windows)
best_models = {}               # Dictionary to store the best model per time series
benchmark_results = {}         # Dictionary to store Naive benchmark error per series

# Metric selected by the user
# List of available evaluation metrics
all_metrics = [ufl.mape, ufl.mse]

# Iterate over each unique time series
for uid in data['unique_id'].unique():
    ts_data = data[data['unique_id'] == uid].copy()
    ts_data = ts_data.sort_values('ds')
    y_values = ts_data['y'].reset_index(drop=True)

    # Skip time series with fewer than 48 observations
    if len(y_values) < min_obs:
        print(f"⏩ Skipping {uid} (only {len(y_values)} observations)\n")
        continue

    # Warn if using MAPE and the series contains zero values
    if metric == 'mape' and (y_values == 0).any():
        warnings.warn(f"⚠️ MAPE selected but series {uid} contains zero values.\n")
    
    models = ts_models[uid]  # Retrieve models assigned to this time series

    # Perform time series cross-validation using rolling windows
    sf = StatsForecast(models=models, freq='MS', n_jobs=1)
    cv_df = sf.cross_validation(
        df=ts_data,
        step_size=rolling_step,
        n_windows=(eval_window - forecast_horizon)//rolling_step + 1,
        h=forecast_horizon
    )

    # Evaluate forecasting accuracy using selected metrics
    results_df = evaluate(
        df=cv_df.drop(columns='cutoff'), 
        train_df=ts_data, 
        metrics=all_metrics
    )

    # Keep only rows corresponding to the selected metric (e.g., 'mape' or 'mse')
    model_errors = results_df[results_df['metric'] == metric]
    model_errors_long = model_errors.melt(
        id_vars=['unique_id', 'metric'], 
        var_name='model', 
        value_name='value'
    )
    avg_errors = model_errors_long.groupby('model')['value'].mean().reset_index()

    # Skip this series if no results were computed
    if avg_errors.empty:
        print(f"⚠️ No evaluation results for {uid}")
        continue

    # Select the best model (excluding Naive)
    non_naive = avg_errors[avg_errors['model'] != 'Naive']
    best_model = non_naive.sort_values('value').iloc[0]

    # Retrieve the error of the Naive model
    naive_error = avg_errors[avg_errors['model'] == 'Naive']['value'].values[0]

    # Store results for this time series
    best_models[uid] = best_model
    benchmark_results[uid] = naive_error

    # Display results
    print(f"✅ Best model for {uid}: {best_model['model']} with avg {metric.upper()} = {best_model['value']:.4f}")
    print(f"📊 Naive benchmark for {uid}: avg {metric.upper()} = {naive_error:.4f}\n\n")


✅ Best model for C1002: RWD with avg MAPE = 0.0803
📊 Naive benchmark for C1002: avg MAPE = 0.0604


✅ Best model for C1016: RWD with avg MAPE = 0.0840
📊 Naive benchmark for C1016: avg MAPE = 0.1214


⏩ Skipping C1050 (only 22 observations)

✅ Best model for C1080: SeasWA with avg MAPE = 0.0692
📊 Naive benchmark for C1080: avg MAPE = 0.0920


✅ Best model for C1126: ARIMA with avg MAPE = 0.1262
📊 Naive benchmark for C1126: avg MAPE = 0.1379




# g) Now forecast into the future and only select the chosen forecast method per time series. Try to make this as efficient as possible.
# h) Create the following outputs as print for each time series: Name of the time series, chosen model, accuracy metric and accuracy value of the chosen forecast method and the Naive as benchmark.
# i) Save the forecasts as .csv file in the directory

In [4]:
# g) Forecast the future using only the best model per series
final_forecasts = []  # List to store final forecasts in Nixtla format

for uid in data['unique_id'].unique():
    if uid not in best_models:
        continue  # Skip if no best model was selected for this series

    ts_data = data[data['unique_id'] == uid].copy()
    ts_data = ts_data.sort_values('ds')  # Ensure correct chronological order

    best_model_name = best_models[uid]['model']
    
    # Find the corresponding model object by its string representation
    model_obj = next((m for m in ts_models[uid] if m.__repr__() == best_model_name), None)
    if model_obj is None:
        print(f"❌ Model {best_model_name} not found for {uid}")
        continue

    # Forecast using only the selected model
    sf = StatsForecast(models=[model_obj], freq='MS', n_jobs=1)
    forecast_df = sf.forecast(df=ts_data, h=forecast_horizon)

    # Ensure consistent format (although unique_id is already set)
    forecast_df['unique_id'] = uid
    final_forecasts.append(forecast_df)

    # h) Print summary of the chosen model and its performance
    print(f"📈 Forecast summary for {uid}:")
    print(f"   ✅ Chosen Model: {best_model_name}")
    print(f"   🔍 Accuracy ({metric.upper()}): {best_models[uid]['value']:.4f}")
    print(f"   🧪 Naive Benchmark ({metric.upper()}): {benchmark_results[uid]:.4f}\n")


# Create summary DataFrame from best_models and benchmark_results
summary_data = []

for uid, model_info in best_models.items():
    summary_data.append({
        'time_series': uid,
        'model': model_info['model'],
        'accuracy': round(model_info['value'], 4),
        'naive_benchmark': round(benchmark_results[uid], 4)
    })

summary_df = pd.DataFrame(summary_data)

# Save the summary table to CSV
summary_df.to_csv(f'forecast_summary_{metric}.csv', index=False)

print("Forecast summary saved as 'forecast_summary.csv'.")

📈 Forecast summary for C1002:
   ✅ Chosen Model: RWD
   🔍 Accuracy (MAPE): 0.0803
   🧪 Naive Benchmark (MAPE): 0.0604

📈 Forecast summary for C1016:
   ✅ Chosen Model: RWD
   🔍 Accuracy (MAPE): 0.0840
   🧪 Naive Benchmark (MAPE): 0.1214

📈 Forecast summary for C1080:
   ✅ Chosen Model: SeasWA
   🔍 Accuracy (MAPE): 0.0692
   🧪 Naive Benchmark (MAPE): 0.0920

📈 Forecast summary for C1126:
   ✅ Chosen Model: ARIMA
   🔍 Accuracy (MAPE): 0.1262
   🧪 Naive Benchmark (MAPE): 0.1379

Forecast summary saved as 'forecast_summary.csv'.


In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import sys
import warnings
import os
import matplotlib.pyplot as plt
from coreforecast.scalers import boxcox, boxcox_lambda, boxcox_inverse
os.environ['NIXTLA_ID_AS_COL'] = '1'

from statsforecast.models import (
    Naive,
    HistoricAverage,
    RandomWalkWithDrift,
    SeasonalNaive,
    SeasonalWindowAverage,
    AutoETS,
    HoltWinters,
    AutoARIMA
)
from statsforecast import StatsForecast
from utilsforecast.evaluation import evaluate
import utilsforecast.losses as ufl

# USER INPUT
file_name = 'PROJECT/proj1_exampleinput.csv'
time_series = ['C1002', 'C1050', 'C1080', 'C1016', 'C3029']
metric = 'mape'  # 'mape' or 'mse'


# Verify if a valid metric was selected
valid_metrics = ['mape', 'mse']
if metric not in valid_metrics:
    raise ValueError(f"Invalid metric. Please choose either 'mape' or 'mse'.")

# a) READ THE DATA
try:
    ds = pd.read_csv(file_name, parse_dates=['Month'])
except FileNotFoundError:
    print(f"Error: The file '{file_name}' was not found.")
    sys.exit(1) # Exit the script if file is not found


# b) Filter only the time series selected by the user, sorting everything by ID and date, renaming the columns so they match what the StatsForecast library expects.


ds = ds[ds['product_class'].isin(time_series)]
ds = ds.sort_values(by=['product_class', 'Month'])
ds = ds.rename(columns={'product_class': 'unique_id', 'Month': 'ds', 'sales_volume': 'y'})
ds = ds.reset_index(drop=True)



#Handling Missing Dates

# Store DataFrames for each unique_id after handling missing dates
processed_dfs = []

# Iterate over each unique product_class (now 'unique_id')
for uid in ds['unique_id'].unique():
    subset_df = ds[ds['unique_id'] == uid].copy()

    # Determine the full date range for this unique_id
    min_date = subset_df['ds'].min()
    max_date = subset_df['ds'].max()
    full_date_range = pd.date_range(start=min_date, end=max_date, freq='MS') # 'MS' for Month Start

    # Create a new DataFrame with the full date range and the unique_id
    # Then merge it with the original data for this unique_id
    reindexed_df = pd.DataFrame({'ds': full_date_range, 'unique_id': uid})
    reindexed_df = pd.merge(reindexed_df, subset_df, on=['unique_id', 'ds'], how='left')

    # Fill 'y' (sales_volume) for missing dates with 0
    missing_dates_count = reindexed_df['y'].isnull().sum()
    if missing_dates_count > 0:
        print(f"Product Class: {uid}")
        missing_dates = reindexed_df[reindexed_df['y'].isnull()]['ds']
        for date in missing_dates:
            print(f"  Missing Date: {date.strftime('%Y-%m-%d')}, Sales Volume filled with: 0")
        reindexed_df['y'] = reindexed_df['y'].fillna(0)
    else:
        print(f"Product Class: {uid} - No implicit missing dates found.")

    processed_dfs.append(reindexed_df)

# Concatenate all processed DataFrames back into a single DataFrame
ds = pd.concat(processed_dfs, ignore_index=True)
ds = ds.sort_values(by=['unique_id', 'ds']).reset_index(drop=True)


#c) MODEL SELECTION BY CATEGORY

# This function helps us figure out if a time series has strong seasonality.
# It first checks if there's enough data to do a proper seasonal decomposition (at least two full cycles).
# If that condition is met, it tries to break the series into three parts: trend, seasonality, and leftover noise (residuals),
# An additive model was selected because it works better when the data has zeros or stays around the same range.
# Then, it compares how much the seasonal part varies compared to the noise.
# If the seasonal variation is clearly stronger (we're using 2x as a threshold), we say the series has high seasonality.
# If there's not enough data or the decomposition fails, we catch the error and just assume there's no strong seasonality.

def high_seasonality(series, period=12):
    if len(series) < 2 * period:
        return False
    try:
        result = seasonal_decompose(series, model='additive', period=period, extrapolate_trend='freq')
        seasonal_var = np.var(result.seasonal.dropna())
        resid_var = np.var(result.resid.dropna())
        return seasonal_var > resid_var * 2
    except Exception as e:
        print(f"Seasonal decomposition failed. Error: {e}")
        return False

#Checks if all values in the series are positive.
def is_positive(series):
    return series.min() > 0

#Checks if the series contains any zero values.
def zeroes_in_dataframe(series):
    return (series == 0).any()

    
#Assigning time series to a category
def get_category(ts_data_for_series, period=12):
    series_values = ts_data_for_series['y']
    if len(series_values) < 2 * period: # Category 1: shorter than two seasonality periods (24 months)
        return 1
    elif zeroes_in_dataframe(series_values) and high_seasonality(series_values, period):
        return 2 # Category 2: zeroes and high seasonality
    elif high_seasonality(series_values, period) and is_positive(series_values):
        return 3 # Category 3: high seasonality and only positive values
    elif not high_seasonality(series_values, period) and zeroes_in_dataframe(series_values):
        return 4 # Category 4: low/no seasonality and zeroes
    else:
        return 5 # Category 5: All remaining cases



category_models = {
    1: [
        # Category 1: Shorter than two seasonality periods
        # Models:
        # - Naive: Serves as a benchmark for all forecasting tasks. It assumes the next value is the last observed one. Useful when data is limited.
        # - HistoricAverage: Calculates the mean of historical values. Useful when no seasonal or trend patterns are observable due to short history.
        # Reasoning: Because there is not enough data, complex models may not be able to capture seasonality or trends optimally. Simple models can work with short ranges.
        
        Naive(),
        HistoricAverage()
        ],
    2: [
        #Category 2: High seasonality + zero values
        # Models:
        # - Naive
        # - SeasonalNaive: Assumes that the current value will be the same as in the same period of the previous season. 
        # - SeasonalWindowAverage: Averages values from the same point in past seasons to smooth out the seasonality. 
        # - SARIMA: A model that can handle both trend and seasonality. It combines autoregression, moving averages, and seasonal differencing to adapt to complex patterns.
        # - Holt-Winters (Additive): Used for series with both trend and seasonality, especially when values can drop to zero. The additive version works better than the multiplicative one in these cases, because it doesn’t break when zeros appear.
        # Reasoning:
        # In this scenario, we have strong seasonal patterns and some zero values in the data. That makes additive models a safer choice, because multiplicative ones can’t handle zeros properly.
        # We use SeasonalNaive and SeasonalWindowAverage because they take advantage of the repeating seasonal patterns. SARIMA and Holt-Winters (Additive) are more flexible, because they can also catch trends along with seasonality.


        Naive(),
        SeasonalNaive(season_length=12),
        SeasonalWindowAverage(window_size=3, season_length=12),
        AutoARIMA(season_length=12, alias='SARIMA'),
        HoltWinters(season_length=12, error_type='A', alias='HW_Add')
        ],
    3: [
        # Category 3: High seasonality + positive values
        # Models:
        # - Naive
        # - SeasonalNaive
        # - SeasonalWindowAverage
        # - SARIMA
        # - AutoETS: Automatically selects between additive and multiplicative exponential smoothing.
        # - Reasoning: With strong seasonality and no zero values, multiplicative models become viable. SeasonalNaive and SeasonalWindowAverage are included because they look at what happened in the same period during past seasons, which helps when patterns repeat over time.
        # SARIMA is included because it is a flexible model that can capture seasonality, trends and complex relationships in the data, such as autocorrelation, moving average effects, and seasonal differencing to remove seasonal trends.
        # AutoETS is a model that picks the best way to handle the data, and since all values are positive, it can also use multiplicative components that adjust better when seasonal effects grow with the data.
        
        Naive(),
        SeasonalNaive(season_length=12),
        SeasonalWindowAverage(window_size=3, season_length=12),
        AutoARIMA(season_length=12, alias='SARIMA'),
        AutoETS(model='ZZZ', alias='AutoETS')],
    4: [
        # Category 4: Low seasonality + zero values
        # Models:
        # - Naive
        # - HistoricAverage
        # - RandomWalkWithDrift: Assumes the series follows a trend with random variation. Useful when there's no seasonality but a clear directional movement.
        # - AutoETS (Simple/Double) Simple Exponential Smoothing assumes no trend/seasonality. Double Exponential Smoothing introduces trend while maintaining simplicity.
        # - ARIMA: General-purpose model that can capture linear trends and autocorrelation without requiring seasonality.
        # - Reasoning: In the absence of strong seasonality, models that focus on level and trend (rather than seasonal cycles) are better.
        # If data is noisy or exhibits random walk behavior, RWWD and ARIMA are excellent choices. AutoETS allows automatic selection between simple and double exponential smoothing.
        # HistoricAverage acts as a fallback for stationary, mean-reverting series.

        Naive(),
        HistoricAverage(),
        RandomWalkWithDrift(),
        AutoETS(model='ZAN', alias='AutoETS_SimpleDouble'),
        AutoARIMA(season_length=1, alias='ARIMA')
        ],
    5: [
        # Category 5: All other cases
        # Models:
        # - Naive
        # - ARIMA: Adaptable model for data with autocorrelation and trend, without requiring seasonality.
        # - AutoETS (Simple/Double, damped trend): Offers flexibility across level and trend modeling with exponential smoothing, including damped trend components for stabilizing long-term forecasts.
        # - HistoricAverage: Provides robustness in noisy datasets or when structure is unclear.
        # - RandomWalkWithDrift: Simple trend model, less prone to overfitting, useful when trend is present but weak.
        # - Reasoning: When the structure of the series is uncertain or doesn't fit neatly into seasonal/non-seasonal classifications, we need versatile models. ARIMA and AutoETS can adapt to various patterns.
        # Damped trends are especially useful when the series appears to trend but might plateau. Simpler models like Naive, HistoricAverage, and RWWD help prevent overfitting and offer interpretability.
        
        Naive(),
        AutoARIMA(season_length=12, alias='ARIMA'),
        AutoETS(model='ZAN', alias='AutoETS_SimpleDouble'),
        HistoricAverage(),
        RandomWalkWithDrift()]
}

# Assign models to each time series based on its category

print("\n---------------------------------------------------- Model Assignment-------------------------------------------------------\n ")

ts_models = {}
for uid in ds['unique_id'].unique():
    ts = ds[ds['unique_id'] == uid]
    cat = get_category(ts)
    ts_models[uid] = category_models[cat]
    print(f"Series {uid} assigned to Category {cat} with models: {[type(m).__name__ for m in ts_models[uid]]}")

print("----------------------------------------------------------------------------------------------------------------------------\n")


# d, e, f) Cross-validation, Evaluation, Best Model Selection
# Define key parameters
forecast_horizon = 12          # Number of periods to forecast (one year)
min_obs = 48                   # Minimum number of observations required for cross-validation (4 years of monthly data for a 1-year forecast window)
rolling_step = 6               # Step size for rolling evaluation (every half year)
eval_window = 24               # Total number of test observations (evaluation period of 2 years)
best_models = {}               # Dictionary to store the best model per time series
benchmark_results = {}         # Dictionary to store Naive benchmark error per series
boxcox_lambdas = {}

# List of available evaluation metrics
all_metrics = [ufl.mape, ufl.mse]

print("\n-------------------------------------------- Cross-Validation and Evaluation -----------------------------------------------\n")


# This part of the code takes care of handling each time series separately. For every unique_id, it grabs all the related data, sorts it by time.

for uid in ds['unique_id'].unique():
    ts_data = ds[ds['unique_id'] == uid].copy()
    ts_data = ts_data.sort_values('ds')
    y_values = ts_data['y'].reset_index(drop=True)

    # Skip time series with fewer than min_obs observations for cross-validation
    if len(y_values) < min_obs:
        print(f"Skipping {uid} (only {len(y_values)} observations, less than {min_obs} required for CV)\n")
        continue

    # Warn if using MAPE and the series contains zero values, as MAPE is undefined for zeros
    if metric == 'mape' and (y_values == 0).any():
        warnings.warn(f"MAPE selected but series {uid} contains zero values. MAPE might be inaccurate or infinite.\n")
    
    models = ts_models[uid]  # Retrieve models assigned to this time series


    #BOX COX TRANSFORMATION FOR TIME SERIES WITHOUT ZERO VALUES
    ts_data_for_cv = ts_data.copy()
    current_lambda = None
    is_series_positive = is_positive(ts_data['y'])
    has_autoarima_model = any(isinstance(m, AutoARIMA) for m in models)

    if is_series_positive and has_autoarima_model:
        try:

            current_lambda = boxcox_lambda(ts_data['y'].values, season_length=12, method='guerrero')
            ts_data_for_cv['y'] = boxcox(ts_data['y'].values, lmbda=current_lambda)
            boxcox_lambdas[uid] = current_lambda
            print(f"Applying Box Cox transformation to'{uid}' with lambda: {current_lambda:.4f}")
        except Exception as exception:
            print(f"Box Cox transformation failed  {uid}. Error: {exception}. Proceeding without transformation.")
            current_lambda = None
            boxcox_lambdas[uid] = None
    else:
        boxcox_lambdas[uid] = None

    # Perform time series cross-validation using rolling windows
    # n_windows calculates how many times we can roll the window over the eval_window period.
    # (+1 because n_windows is inclusive of the first window)
    n_windows_calculated = (eval_window - forecast_horizon) // rolling_step + 1
    
    try:
        sf = StatsForecast(models=models, freq='MS', n_jobs=1) # freq='MS' for monthly start
        cv_df = sf.cross_validation(
            df=ts_data,
            step_size=rolling_step,
            n_windows=n_windows_calculated,
            h=forecast_horizon
        )
        
        cv_df = cv_df.reset_index()
    except Exception as error:
        print(f"Cross-validation failed for {uid}. Error: {error}\n")
        continue

    #BOX COX INVERSE
    if current_lambda is not None:
        for model_obj in models:
            model_col_name = model_obj.alias if hasattr(model_obj, 'alias') else type(model_obj).__name__
            if isinstance(model_obj, AutoARIMA) and model_col_name in cv_df.columns:
                try:
                    cv_df[model_col_name] = boxcox_inverse(cv_df[model_col_name].values, lmbda=current_lambda)
                    cv_df[model_col_name] = cv_df[model_col_name].clip(lower=0)
                except Exception as exception:
                    print(f"Inverse Box Cox Failed {uid} (model {model_col_name}). Error: {exception}")
    

    # Evaluate forecasting accuracy using selected metrics
    # Drop 'cutoff' column as it's not needed for evaluation
    results_df = evaluate(
        df=cv_df.drop(columns='cutoff'), 
        train_df=ts_data, # train_df is needed for some metrics like MAPE
        metrics=all_metrics
    )

    # Filter evaluation results to keep only the user-selected metric
    model_errors = results_df[results_df['metric'] == metric]
    
    # Melt the DataFrame to easily group by model and calculate average errors
    model_errors_long = model_errors.melt(
        id_vars=['unique_id', 'metric'], 
        var_name='model', 
        value_name='value'
    )
    # Calculate the average error for each model across all windows
    avg_errors = model_errors_long.groupby('model')['value'].mean().reset_index()

    # Skip this series if no evaluation results were computed (e.g., if model failed)
    if avg_errors.empty:
        print(f"No evaluation results for {uid} after filtering by metric. Skipping.\n")
        continue

    # Select the best model (excluding Naive from the "best model" selection, but keeping it for benchmark)
    non_naive_models = avg_errors[avg_errors['model'] != 'Naive']
    best_model = non_naive_models.sort_values('value').iloc[0]

    # Retrieve the error of the Naive model for benchmarking
    naive_error = avg_errors[avg_errors['model'] == 'Naive']['value'].values[0]

    # Store results for this time series
    best_models[uid] = best_model
    benchmark_results[uid] = naive_error

    # Display results for each time series
    print(f"Best model for {uid}: {best_model['model']} with avg {metric.upper()} = {best_model['value']:.4f}")
    print(f"Naive benchmark for {uid}: avg {metric.upper()} = {naive_error:.4f}\n")
print("----------------------------------------------------------------------------------------------------------------------------\n")


#g, h, i) Forecast into the future and Save Outputs
final_forecasts_list = []

print("\n-------------------------------------------- Final Forecasting and Summary -------------------------------------------------\n")
# Iterate through each unique time series ID present in the data.
for uid in ds['unique_id'].unique():
    # Check if a best model was selected for the current time series (uid) during cross-validation.
    if uid not in best_models:
        print(f"Skipping final forecast for {uid} (was skipped during CV/evaluation).\n")
        continue # Move to the next time series.

    # Extract the data for the current time series and ensure it's sorted chronologically.
    ts_data = ds[ds['unique_id'] == uid].copy().sort_values('ds')

    # Retrieve the information about the best performing model for this time series.
    best_model_info = best_models[uid]
    best_model_name = best_model_info['model']
    
    # Find the model from the ts_models
    model_obj = next((m for m in ts_models[uid] if m.__repr__() == best_model_name), None)
    if model_obj is None:
        print(f"Model object for {best_model_name} not found for {uid}. Cannot forecast.")
        continue # Move to the next time series if the model object can't be found.
    
    #Box Cox Transformation
    ts_data_for_forecast = ts_data.copy()
    current_lambda = boxcox_lambdas.get(uid)
    is_best_model_autoarima = isinstance(model_obj, AutoARIMA)
    if is_best_model_autoarima and current_lambda is not None:
        try:
            ts_data_for_forecast['y'] = boxcox(ts_data['y'].values, lmbda=current_lambda)
            print(f"Aplicando transformación Box-Cox a los datos de entrada para la previsión final de '{uid}' con el mejor modelo '{best_model_name}'.")
        except Exception as e:
            print(f"Advertencia: La transformación Box-Cox para la entrada de la previsión final falló para {uid}. Error: {e}. Procediendo con los datos originales.")
            ts_data_for_forecast = ts_data.copy() # Vuelve a los datos originales si la transformación falla.
    # FIN DE MODIFICACIÓN

    # Final forecast using only the selected best model.
    try:
        sf = StatsForecast(models=[model_obj], freq='MS', n_jobs=1)
        forecast_df = sf.forecast(df=ts_data, h=forecast_horizon).reset_index()
        if is_best_model_autoarima and current_lambda is not None:
            forecast_col_name = model_obj.alias if hasattr(model_obj, 'alias') else type(model_obj).__name__
            if forecast_col_name in forecast_df.columns:
                try:
                    forecast_df[forecast_col_name] = boxcox_inverse(forecast_df[forecast_col_name].values, lmbda=current_lambda)
                    forecast_df[forecast_col_name] = forecast_df[forecast_col_name].clip(lower=0)
                    print(f"Aplicada la transformación inversa Box-Cox a los pronósticos finales para '{uid}'.")
                except Exception as e:
                    print(f"Advertencia: La transformación inversa Box-Cox para los pronósticos finales falló para {uid}. Error: {e}")
    except Exception as error:
        print(f"Error during final forecast for {uid}, model {best_model_name}: {error}")
        continue

    
    print(f"Forecast summary for {uid}:")
    print(f"   Chosen Model: {best_model_name}")
    print(f"   Accuracy ({metric.upper()}): {best_model_info['value']:.4f}")
    print(f"   Naive Benchmark ({metric.upper()}): {benchmark_results[uid]:.4f}\n")

# i) Save the forecasts as .csv file

if final_forecasts_list:
    current_dir = os.path.dirname(os.path.abspath(__file__))
    file_path = os.path.join(current_dir, f'future_forecasts_{metric}.csv')
    pd.concat(final_forecasts_list, ignore_index=True).to_csv(file_path, index=False)
    print(f"Forecasts saved as '{file_path}'.")
else:
    print("No future forecasts were generated or saved.")
print("----------------------------------------------------------------------------------------------------------------------------\n")



ImportError: cannot import name 'boxcox_inverse' from 'coreforecast.scalers' (/opt/anaconda3/lib/python3.12/site-packages/coreforecast/scalers.py)